# cjm-fasthtml-workers

> Background worker system for FastHTML with multiprocess job execution, cancellation support, and streaming capabilities.

## Install

```bash
pip install cjm_fasthtml_workers
```